In [1]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize

np.set_printoptions(suppress=True) # supress scientific notation


In [2]:
## parâmetros

def pars(st=3):
    global rho, beta, theta, eta, psi, kappa, lamb, gamma1, G, phi
    
    np.random.seed(11)
    #a = np.random.rand(st)
    #alfa = a/np.sum(a)
    
    rho = 0.19
    beta = 2

    theta = 3.44
    eta = 0.25
    psi = eta**eta*(1-eta)**(1-eta)
    kappa = 1/(1 - eta)

    lamb = (theta*(1 - eta) - 1 ) / (theta*(1- eta)) 
    gamma1 = eta*kappa
    
    G = gamma(   1 - np.multiply( np.divide(1, np.multiply(theta, (1-rho)) ), np.divide(1, (1 - eta) ) ) )   

    phi = np.random.rand(st)
    #tau = np.random.rand(st)

  
    


In [89]:

def taus2(st):
        
    tau = np.random.uniform(low=-0.99, high=1000, size=(st))
    tau[0] = 100

    alfa = np.random.uniform(low=0, high=1, size=(st))
    alfa = alfa/np.sum(alfa)
    
    x1 = np.array([tau, alfa])
    
    return x1
st = 10
x1 = taus2(st)

In [4]:
# Tempo Gasto na escola

def sf( ):
    global s
    s = np.power( (1+ np.divide( (1-eta), ( np.multiply(beta, phi) ) ) ), -1 )
    #s = s.reshape(st, 1)
    return s



In [5]:
## N_i = H_i / w_i^gamma1

def N_if(x1):
    B = np.power((1 + x1[0]), -eta)
    S = np.multiply(np.power(s, phi), np.power((1 - s), ((1-eta)/beta) ) )  
    
    A_i = np.power(np.multiply(psi, np.multiply(B, S) ), theta*lamb )
    
    C = np.power( (1+ x1[0]), -gamma1 ) 
    D = np.multiply(np.power( s, (phi*(1-gamma1)) ), np.power(eta, gamma1) ) 
    
    N_i = np.multiply(np.multiply(np.multiply(C, D), G ), A_i )
    return N_i
    


In [6]:
#  sistema linear

def lin_sys(x1):
    A = np.repeat(x1[1], st).reshape(st, st).T  # matriz de parâmetros
    
    B = np.repeat(x1[1], st).reshape(st, st).T        # Matriz de alfas 

    for i in range(st):
        for j in range(st):
            if i==j:
                A[i, j] = 1 - (A[i, j] - 1)*(theta*lamb + gamma1)
                B[i, j] = B[i, j] - 1
            if i != j:
                A[i, j] = A[i, j]*(theta*lamb + gamma1) 
            if j != 0:
                A[i,j] = -1*A[i, j]
    
    N_i = np.log( N_if(x1) )     # Logaritmo de N_i
    C = np.log(x1[1])           # Logaritmo dos alfas
    D = np.dot(B, N_i) + C     # produto matricial  
    
    sol = np.linalg.solve(A, D)    # Resolvendo o sistema  A w = D

    return sol


In [71]:
# solver para o sistema linear

def solver(x1, st):
    global w
    w_n = ["w" + str(i) for i in range(1,st+1)]
    pars(st)
    s = sf()
    w = np.exp(lin_sys(x1))
    print('Salários')
    for i in range(st):
        print(f'{w_n[i]} = { np.round(w[i], 4) }')
    N_i = N_if(x1)
    return w

In [78]:
st = 10
x1 = taus2(st)
solver(x1, st)



Salários
w1 = 0.8282
w2 = 1.9445
w3 = 1.9825
w4 = 1.4417
w5 = 1.0951
w6 = 2.2319
w7 = 4.8657
w8 = 1.043
w9 = 1.2748
w10 = 0.8875


array([0.82823025, 1.94451555, 1.98247029, 1.44174775, 1.09513458,
       2.231942  , 4.86570162, 1.04302542, 1.2748244 , 0.88745556])

In [9]:
# capital humano de cada setor
N_i = N_if(x1)

H_i = np.multiply(N_i, np.power(w, gamma1) ) 

In [10]:
# produto da economia

Y = np.prod(np.power(H_i, x1[1] ))

In [11]:
def w_til_f(x1):
    A = np.multiply(np.power((1 + x1[0]), -eta), w )
    S = np.multiply(np.power(s, phi), np.power((1 - s), ((1-eta)/beta) ) )
    
    w_til = np.multiply(psi, np.multiply(A, S) )
    
    return w_til


In [12]:
def p_i_f(x1):
    w_til = w_til_f(x1)
    p_i =  np.power(w_til, theta)/ np.sum( np.power(w_til, theta) ) 
    
    return p_i
    

In [13]:
def W_i_f(x1):
    w_til = w_til_f(x1)

    S_wtil = np.power((np.sum( np.power(w_til, theta) ) ), (kappa/theta) )
    S = np.power((1-s), (-1/beta))
    W_i = 1/kappa*np.multiply(np.multiply(S, S_wtil), G ) 
    
    return W_i
    
    

In [14]:
W_i_f(x1)

array([4.8222433 , 4.0644989 , 5.92482821, 6.7870404 , 5.77080375,
       6.00280416, 4.02986311, 6.0095833 , 7.42605465, 7.16483302])

In [84]:
## Dados fake

np.random.seed(20)
a = np.random.rand(st)
p_s = a/np.sum(a)
p_s = p_s[1:st]

W_s = np.random.uniform(low=0.05, high=10, size=(st))





In [90]:

def obj2(x1):       
    x1 = x1.reshape((2, st)) 
    x1[0, 0] = 1000
#    x1[1] = x1[1]/np.sum(x1[1])
   
    W_i = W_i_f(x1) 
    p_i = p_i_f(x1)   
    p_i = p_i[1:st]
    
    D =   (np.power(np.divide( (W_i - W_s), W_s ), 2)).sum() + (np.power(np.divide( (p_i - p_s), p_s ), 2)).sum() 
    D = np.log(D)    
    return D


In [91]:
obj2(x1)

5.307539855469244

In [92]:
Bd = ((-0.99, 100000), )*st + ((0, 1), )*st
Bd = np.array(Bd, dtype=object)



In [93]:
x1 = taus2(st)

cc = 0
def callback(x):
    global cc
    cc += 1
    fobj = obj2(x)
    print(f'\033[1;033mObjetivo: {np.around(fobj, 5)}, iter: {cc}') 

%time sol= minimize(obj2, x1,  method='L-BFGS-B', callback=callback, bounds=Bd, tol=1e-100, options={'maxiter':1e4, 'maxfun':1e10000,  'maxcor': 1000, 'eps': 1e-9,})
 

Objetivo: 4.81011, iter: 1
Objetivo: 3.73801, iter: 2
Objetivo: 3.71357, iter: 3
Objetivo: 3.7032, iter: 4
Objetivo: 3.70153, iter: 5
Objetivo: 3.7013, iter: 6
Objetivo: 3.7013, iter: 7
Objetivo: 3.70129, iter: 8
Objetivo: 3.70127, iter: 9
Objetivo: 3.70123, iter: 10
Objetivo: 3.70111, iter: 11
Objetivo: 3.70078, iter: 12
Objetivo: 3.69992, iter: 13
Objetivo: 3.69766, iter: 14
Objetivo: 3.69149, iter: 15
Objetivo: 3.67642, iter: 16
Objetivo: 3.6435, iter: 17
Objetivo: 3.59221, iter: 18
Objetivo: 3.55597, iter: 19
Objetivo: 3.52351, iter: 20
Objetivo: 3.49148, iter: 21
Objetivo: 3.34553, iter: 22
Objetivo: 3.34537, iter: 23
Objetivo: 3.07829, iter: 24
Objetivo: 3.01315, iter: 25
Objetivo: 2.86787, iter: 26
Objetivo: 2.84127, iter: 27
Objetivo: 2.5785, iter: 28
Objetivo: 2.51619, iter: 29
Objetivo: 2.51539, iter: 30
Objetivo: 2.48709, iter: 31
Objetivo: 2.46043, iter: 32
Objetivo: 2.45225, iter: 33
Objetivo: 2.43535, iter: 34
Objetivo: 2.39716, iter: 35
Objetivo: 2.38098, iter: 36
Objeti

In [94]:
sol.x

array([  100.        ,   156.18716982,    22.30459847,     3.36067444,
         116.32135204,    48.35837082, 19917.31308162,     2.37483195,
           1.54283635,     2.04611204,     0.09777858,     0.14630357,
           0.13804288,     0.13608277,     0.05339823,     0.10890253,
           0.08167879,     0.10212094,     0.04155527,     0.09413643])

In [ ]:

x0 = np.ndarray.flatten(x1)

const =  {'type': 'eq', 'fun': lambda x0: np.sum(x0[(st+1):]) - 1}

sol = minimize(obj2, x0,  method='trust-constr', bounds=Bd,
         constraints= const,
         options={'maxiter':1e5, 'verbose': 2,
                 'xtol': 1e-18,
                  'gtol': 1e-18})


In [96]:
x0 = np.ndarray.flatten(x1)

sol = minimize(obj2, x0,  method='SLSQP', bounds=Bd,
         constraints= const,
         options={'maxiter':1e5}) 

In [97]:
sol


     fun: 3.7016221954091417
     jac: array([ 0.        , -0.0003126 ,  0.00000194,  0.00000054,  0.0000014 ,
        0.00000671, -0.00046504,  0.00000125,  0.00000033,  0.00000626,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])
 message: 'Optimization terminated successfully'
    nfev: 515
     nit: 23
    njev: 23
  status: 0
 success: True
       x: array([100.        ,   1.48158152, 707.80388326, 620.04849408,
       777.58249602, 458.79840536, 382.28683738, 291.1780597 ,
       556.78667344,  83.20750439,   0.09777858,   0.15716786,
         0.14890717,   0.14694706,   0.06426252,   0.11976682,
         0.09254308,   0.11298523,   0.05241955,   0.10500072])